In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

import plotly.graph_objects as go

In [2]:
path = '../Data/Preprocessed/drop_category.csv'

df_origin = pd.read_csv(path)

innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_6052\3324504427.py:3: DtypeWarning: Columns (12,15,16,17,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_origin = pd.read_csv(path)
C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_6052\3324504427.py:5: DtypeWarning: Columns (4,5,6,7,20,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  innerOut_df_origin = pd.read_csv('../Data/Preprocessed/innerOutTaged.csv')


In [3]:
innerOut_df_origin.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,bigPath,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx
0,2022-08-01 14:30:55+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=24596,jobs,NaN,NaN,NaN,NaN,etc,etc,...,채용/인재,mcd=c01,NaN,NaN,NaN,시공/감리/생산,메인,jobse,NaN,idx=24596
1,2022-08-01 14:32:14+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=23934,jobs,NaN,NaN,NaN,NaN,etc,etc,...,채용/인재,mcd=c01,NaN,NaN,NaN,시공/감리/생산,메인,jobse,NaN,idx=23934


In [4]:
df_origin.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx,category_small
0,2023-07-11 00:58:56+09:00,111.90.187.156,/index.html,etc,NaN,http://www.lafent.com,NaN,lafent,etc,etc,...,NaN,NaN,NaN,NaN,메인,메인,index.html,NaN,NaN,메인//메인//메인
1,2023-07-11 15:29:14+09:00,112.164.156.215,/sns/qna_view.html?mcd=b01&bsp=q01&idx=5245,social media,NaN,NaN,NaN,NaN,etc,etc,...,mcd=b01,NaN,NaN,bsp=q0,조경인광장,조경인Q&A,sns,NaN,idx=5245,커뮤니티//조경인광장//조경인Q&A


### bigPath 기준으로 전처리

In [62]:
tmp01 = df_origin.groupby(['New IP'])['bigPath'].nunique().reset_index().query('bigPath > 1')['New IP']
tmp02 = df_origin.loc[df_origin['New IP'].isin(tmp01)]

In [63]:
tmp02.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx,category_small
188002,2022-08-01 14:36:51+09:00,1.215.17.66,/jobse/job_view.html?mcd=a08&idx=24766,jobs,NaN,NaN,NaN,NaN,etc,etc,...,mcd=a08,NaN,NaN,NaN,전체채용,서울,jobse,NaN,idx=24766,채용/인재//전체채용//서울
188003,2022-08-01 15:31:08+09:00,1.215.17.66,/news/view.html?news_id=114353,mobile news,NaN,https://search.naver.com/search.naver?sm=tab_h...,행복도시 생활권별 가로수,naver,etc,etc,...,NaN,NaN,NaN,NaN,뉴스홈,뉴스홈,news,news_id=114353,NaN,뉴스//뉴스홈//뉴스홈


In [68]:
def drop_duplicates_02(x):
    value = x.loc[:, 'bigPath']
    value.index = range(len(value))
    value_copy = value.iloc[1:]

    value_copy = pd.DataFrame({'old' : value_copy})
    value = pd.DataFrame({'new' : value})

    value_copy = pd.concat([value_copy, pd.DataFrame({'old' : ['end']})], axis = 0)
    value_copy.index = range(len(value_copy))

    concat = pd.concat([value, value_copy], axis = 1, ignore_index = True)
    concat.columns = ['new', 'old']

    index = concat.loc[concat.apply(lambda x : x['new'] != x['old'], axis = 1)].index
    result = x.iloc[index]

    result.to_csv('../Data/Preprocessed/drop_category_big.csv',
                  encoding = 'utf-8-sig',
                  index = False,
                  mode = 'a',
                  escapechar = '\\')


In [69]:
tmp02.groupby(['New IP']).progress_apply(lambda x : drop_duplicates_02(x))

100%|██████████| 78138/78138 [03:24<00:00, 381.26it/s]


""


In [76]:
df_over = pd.read_csv('../Data/Preprocessed/drop_category_big.csv')
df_over = df_over.loc[df_over['New IP'] != 'New IP']

In [79]:
df_over.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx,category_small
0,2023-07-11 19:01:18+09:00,110.93.154.210,/inews/news_view.html?news_id=131900,news,NaN,NaN,NaN,NaN,etc,etc,...,NaN,NaN,NaN,NaN,뉴스홈,뉴스홈,inews,news_id=131900,NaN,뉴스//뉴스홈//뉴스홈
1,2023-07-11 21:06:10+09:00,110.93.154.210,/magazine/atc_view.html?news_id=5739&gbn=02,magazine,NaN,NaN,NaN,NaN,etc,etc,...,NaN,NaN,NaN,NaN,매거진홈,매거진홈,magazine,news_id=5739,NaN,매거진//매거진홈//매거진홈


In [81]:
df_over = df_over[['New IP', 'User Type', 'bigPath']]

In [82]:
df_over.head(2)

,New IP,User Type,bigPath
0,1000201,내부,뉴스
1,1000201,내부,매거진


In [122]:
def makeData(x):
    value = x.loc[:, 'bigPath']
    value.index = range(len(value))
    value_copy = value.iloc[1:]
    value_copy.index = range(len(value_copy))
    value = value.iloc[:-1]

    value_copy = pd.DataFrame({'end' : value_copy})
    value = pd.DataFrame({'start' : value})



    concat = pd.concat([value, value_copy], axis = 1, ignore_index = True)
    concat['New IP'] = np.nan
    concat['User Type'] = np.nan

    concat['New IP'] = x['New IP'].values[0]
    concat['User Type'] = x['User Type'].values[0]

    concat.to_csv('../Data/Preprocessed/sankeyDiagram.csv',
                  encoding = 'utf-8-sig',
                  mode = 'a',
                  header = None,
                  index = False)

In [123]:
df_over.groupby(['User Type', 'New IP']).progress_apply(lambda x : makeData(x))

100%|██████████| 83398/83398 [03:12<00:00, 434.01it/s]


""


In [138]:
sankey_df = pd.read_csv('../Data/Preprocessed/sankeyDiagram.csv', header = None)
sankey_df.columns = ['start', 'end', 'New IP', 'User Type']

In [139]:
key_dict = {}

for i, value in enumerate(sankey_df['start'].unique()):
    key_dict[value] = i


In [140]:
sankey_df['start'] = sankey_df['start'].apply(lambda x : key_dict[x])
sankey_df['end'] = sankey_df['end'].apply(lambda x : key_dict[x])

sankey_df = sankey_df.groupby(['User Type', 'start', 'end'])['New IP'].nunique().reset_index()
inner = sankey_df.loc[sankey_df['User Type'] == '내부']
out = sankey_df.loc[sankey_df['User Type'] == '외부']

In [155]:
list(key_dict.keys())

['뉴스',
 '채용/인재',
 '검색',
 '기타',
 '메인',
 '작품/조경수/자재',
 '포토갤러리',
 'e-아카데미',
 '매거진',
 '커뮤니티',
 '고객센터',
 '마이페이지',
 '인기통합정보']

In [149]:
inner.head(2)

,User Type,start,end,New IP
0,내부,0,1,6528
1,내부,0,2,650


In [158]:
# Define the nodes and links for the Sankey chart
nodes = list(key_dict.keys())

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source = inner['start'].values,
        target = inner['end'].values,
        value = inner['New IP'].values
    )
))

# Customize the layout and appearance
fig.update_layout(title_text="Sankey Diagram Example")
fig.show()

### bigPath가 하나인 사람

In [5]:
df_one = df_origin.groupby(['New IP'])['bigPath'].nunique().reset_index().query('bigPath == 1')['New IP']
df_one = df_origin.loc[df_origin['New IP'].isin(df_one)]
df_one = df_one.groupby(['New IP', 'User Type'])['bigPath'].unique().reset_index()
df_one['bigPath'] = df_one['bigPath'].apply(lambda x : x[0])

In [6]:
# tmp01 = df_one.groupby(['User Type', 'bigPath'])['New IP'].count().reset_index()//
tmp01 = df_one.groupby(['User Type', 'bigPath'])['New IP'].unique().reset_index()
tmp01 = tmp01.loc[tmp01['bigPath'] != 'bigPath']
tmp01['num_New IP'] = tmp01['New IP'].apply(lambda x : len(x))
tmp_total = df_origin.groupby(['User Type'])['New IP'].nunique().reset_index()
tmp_total = tmp_total.loc[tmp_total['User Type'] != 'User Type']
tmp_total = tmp_total.rename(columns = {'New IP' : 'total_New IP'})

tmp01 = tmp01.merge(tmp_total, on = ['User Type'], how = 'left')
tmp01['단일 카테고리 비율'] = round(tmp01['num_New IP'] / tmp01['total_New IP'] * 100, 2)

In [7]:
tmp01

,User Type,bigPath,New IP,num_New IP,total_New IP,단일 카테고리 비율
0,내부,e-아카데미,"[6515, 6532, 17212, 18548, 20216, 21535, 21871...",132,215350,0.06
1,내부,검색,"[6655, 10217, 12397, 12719, 13061, 18936, 2467...",714,215350,0.33
2,내부,고객센터,"[1276, 2054, 2056, 4733, 4792, 5029, 5092, 509...",3598,215350,1.67
3,내부,기타,"[612, 653, 904, 1049, 1133, 1257, 1271, 1326, ...",14439,215350,6.70
4,내부,뉴스,"[37, 38, 40, 46, 47, 49, 51, 81, 639, 654, 667...",45911,215350,21.32
5,내부,마이페이지,"[3272, 4206, 6523, 7201, 9271, 10296, 11719, 1...",708,215350,0.33
6,내부,매거진,"[356, 1277, 1278, 1319, 2058, 3285, 3471, 3492...",6232,215350,2.89
7,내부,메인,"[36, 39, 45, 48, 97, 647, 1168, 1189, 1267, 17...",7714,215350,3.58
8,내부,인기통합정보,"[3262, 3329, 3469, 3478, 3479, 3480, 3495, 349...",8205,215350,3.81
9,내부,작품/조경수/자재,"[834, 3475, 3507, 4148, 4164, 4191, 4302, 4613...",7139,215350,3.32


In [78]:
innerOut_df_origin.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,bigPath,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx
0,2022-08-01 14:30:55+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=24596,jobs,NaN,NaN,NaN,NaN,etc,etc,...,채용/인재,mcd=c01,NaN,NaN,NaN,시공/감리/생산,메인,jobse,NaN,idx=24596
1,2022-08-01 14:32:14+09:00,1.220.249.83,/jobse/job_view.html?mcd=c01&idx=23934,jobs,NaN,NaN,NaN,NaN,etc,etc,...,채용/인재,mcd=c01,NaN,NaN,NaN,시공/감리/생산,메인,jobse,NaN,idx=23934


In [44]:
result = {'User Type' : [],
          'bigPath' : [],
          'count_New IP' : []}

for i, row in tqdm(tmp01[['User Type', 'bigPath', 'New IP']].iterrows()):
    tmp = innerOut_df_origin.loc[(innerOut_df_origin['User Type'] == row['User Type']) & (innerOut_df_origin['New IP'].isin(row['New IP']))]
    result['count_New IP'].append(tmp['New IP'].nunique())
    result['bigPath'].append(row['bigPath'])
    result['User Type'].append(row['User Type'])

result = pd.DataFrame(result)

tmp_total = innerOut_df_origin.groupby(['User Type'])['New IP'].nunique().reset_index().rename(columns = {'New IP' : 'total_New IP'})

result = result.merge(tmp_total, on = 'User Type', how = 'left')

26it [00:09,  2.84it/s]


In [45]:
tmp_total = innerOut_df_origin.groupby(['User Type'])['New IP'].nunique().reset_index().rename(columns = {'New IP' : 'total_New IP'})
result = result.merge(tmp_total, on = 'User Type', how = 'left')

result['단일 카테고리 이용자 비율'] = round(result['count_New IP'] / result['total_New IP'] * 100, 2)

complex_user = result.groupby(['User Type'])['count_New IP'].sum().reset_index()
complex_user = complex_user.merge(tmp_total, on = ['User Type'], how = 'left')
complex_user['복합 카테고리 이용자 비율'] = 100 - round(complex_user['count_New IP'] / complex_user['total_New IP'] * 100, 2)

KeyError: 'total_New IP'

In [55]:
df_over = pd.read_csv('../Data/Preprocessed/drop_category_big.csv', on_bad_lines = "skip")

C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_20448\726781915.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_over = pd.read_csv('../Data/Preprocessed/drop_category_big.csv', on_bad_lines = "skip")


In [56]:
df_over.head(2)

,Timestamp,IP,URL,Category,URL Search Word,Referer,Referer Search Word,Referer Source,Referer Category,Referer Content Type,...,bigPath,mcd,rk,bt,bsp,middlePath,smallPath,tmp,news_id,idx
0,2023-07-11 00:58:56+09:00,111.90.187.156,/index.html,etc,NaN,http://www.lafent.com,NaN,lafent,etc,etc,...,메인,NaN,NaN,NaN,NaN,메인,메인,index.html,NaN,NaN
1,2023-07-11 15:29:14+09:00,112.164.156.215,/sns/qna_view.html?mcd=b01&bsp=q01&idx=5245,social media,NaN,NaN,NaN,NaN,etc,etc,...,커뮤니티,mcd=b01,NaN,NaN,bsp=q0,조경인광장,조경인Q&A,sns,NaN,idx=5245


In [59]:
df_over.groupby(['New IP'])['bigPath'].nunique().reset_index()['bigPath'].value_counts()
# .query('bigPath != 1')['New IP']

bigPath
1    187988
Name: count, dtype: int64

In [5]:
import plotly.graph_objects as go

# Define the nodes and links for the Sankey chart
nodes = ["Node A", "Node B", "Node C", "Node D"]
links = [
    {"source": 0, "target": 1, "value": 20},
    {"source": 0, "target": 2, "value": 10},
    {"source": 1, "target": 3, "value": 15},
    {"source": 2, "target": 3, "value": 5},
]

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=nodes
    ),
    link=dict(
        source=[link["source"] for link in links],
        target=[link["target"] for link in links],
        value=[link["value"] for link in links]
    )
))

# Customize the layout and appearance
fig.update_layout(title_text="Sankey Diagram Example")
fig.show()